In [40]:
from __future__ import print_function
import cv2
import os
import glob
import numpy as np


import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import StratifiedShuffleSplit

In [64]:
def processCategory(category):
    y = []
    if category == 'angry':
        y = [1, 0, 0, 0, 0, 0, 0]        
    elif category == 'disgust':
        y = [0, 1, 0, 0, 0, 0, 0]
    elif category == 'fear':
        y = [0, 0, 1, 0, 0, 0, 0]
    elif category == 'happy':
        y = [0, 0, 0, 1, 0, 0, 0]
    elif category == 'neutral':
        y = [0, 0, 0, 0, 1, 0, 0]
    elif category == 'sad':
        y = [0, 0, 0, 0, 0, 1, 0]
    elif category == 'surprise':
        y = [0, 0, 0, 0, 0, 0, 1]
    
    return y

def load_data(categories):
    # Training and Testing
    data             = []
    target           = []
    size_image       = 32
    img_dir          = 'train'    
    
    for idx, category in enumerate(categories):
        img_dir_aux = img_dir + '/'+ category
        data_path   = os.path.join(img_dir_aux, '*g')
        files       = glob.glob(data_path)
        for f1 in files:
            img        = cv2.imread(f1)
            img_resize = cv2.resize(img, (size_image, size_image), interpolation = cv2.INTER_AREA)
            data.append(img_resize)
            target.append(idx)

    target     = keras.utils.to_categorical(target, len(categories))    
    data       = np.array(data)
    target     = np.array(target)
    stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.8, random_state=42)
    for test_idx, train_idx in stratSplit.split(data, target):        
        train_data   = data[train_idx]
        train_target = target[train_idx]                        
        test_data    = data[test_idx]
        test_target  = target[test_idx]
        
    # Validation
    val_data   = []
    val_target = []
    img_dir    = 'val'
    for idx, category in enumerate(categories):        
        img_dir_aux = img_dir + '/'+ category
        data_path   = os.path.join(img_dir_aux, '*g')
        files       = glob.glob(data_path)        
        for f1 in files:
            img        = cv2.imread(f1)
            img_resize = cv2.resize(img, (size_image, size_image), interpolation = cv2.INTER_AREA)
            val_data.append(img_resize)
            val_target.append(idx)
    
    val_target   = keras.utils.to_categorical(val_target, len(categories))                
    val_data     = np.array(val_data)
    val_target   = np.array(val_target)
    
    np.random.shuffle(val_data)
    np.random.shuffle(val_target)    
    
    return train_data, train_target, test_data, test_target, val_data, val_target

In [63]:
categories  = ['angry','disgust','fear','happy','neutral','sad','surprise']
train_data, train_target, test_data, test_target, val_data, val_target = load_data(categories)


[0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 1.]
[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 1

In [76]:
batch_size        = 32
num_classes       = 7
data_augmentation = True
num_predictions   = 20
save_dir          = os.path.join(os.getcwd(), 'saved_models')
model_name        = 'faces.h5'
categories  = ['angry','disgust','fear','happy','neutral','sad','surprise']
train_data, train_target, test_data, test_target, val_data, val_target = load_data(categories)

In [77]:
epochs            = 100

In [78]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=train_data.shape[1:]))
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))


model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

datagen.fit(train_data)

# Fit the model on the batches generated by datagen.flow().
generator = datagen.flow(train_data, train_target,batch_size=batch_size)
model.fit_generator(generator, epochs=epochs,validation_data=(test_data, test_target),workers=4, steps_per_epoch = len(generator))

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(test_data, test_target, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


model.summary()

Epoch 1/100
73/73 [==============================] - 8s 109ms/step - loss: 13.8437 - acc: 0.1396 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 2/100
73/73 [==============================] - 7s 90ms/step - loss: 13.8827 - acc: 0.1387 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 3/100
73/73 [==============================] - 7s 94ms/step - loss: 13.8780 - acc: 0.1390 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 4/100
73/73 [==============================] - 7s 94ms/step - loss: 13.8803 - acc: 0.1388 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 5/100
73/73 [==============================] - 7s 94ms/step - loss: 13.8780 - acc: 0.1390 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 6/100
73/73 [==============================] - 7s 96ms/step - loss: 13.8825 - acc: 0.1387 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 7/100
73/73 [==============================] - 7s 93ms/step - loss: 13.8871 - acc: 0.1384 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 8/100
73/73 [==============================] - 7s 98ms/

Epoch 62/100
73/73 [==============================] - 7s 90ms/step - loss: 13.8803 - acc: 0.1388 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 63/100
73/73 [==============================] - 7s 91ms/step - loss: 13.8825 - acc: 0.1387 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 64/100
73/73 [==============================] - 7s 95ms/step - loss: 13.8803 - acc: 0.1388 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 65/100
73/73 [==============================] - 7s 93ms/step - loss: 13.8825 - acc: 0.1387 - val_loss: 13.8749 - val_acc: 0.1392
Epoch 66/100
72/73 [============================>.] - ETA: 0s - loss: 13.9005 - acc: 0.1376

KeyboardInterrupt: 